In [15]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.nn import CrossEntropyLoss

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can use other GPT-2 variants as well
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Add the [PAD] token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))


# Define your input texts as a list
input_texts = "Once upon a time, there was a" 
# [
    # "Once upon a time, there was a",
    # "In a galaxy far, far away, there were",
    # Add more input texts as needed
# ]

# Tokenize the input texts and convert them to tensors
input_ids = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)['input_ids']

# Shift the input_ids to the right by one position to create target_ids
# target_ids = input_ids[:, 1:]

# Generate logits from the model
with torch.no_grad():
    outputs = model(input_ids=input_ids, labels =  input_ids)
    logits = outputs.logits
    loss = outputs.loss
    
# # Reshape target_ids to match the shape of logits
# target_ids = target_ids.reshape(-1)

# # Calculate the CrossEntropyLoss
# loss_fn = CrossEntropyLoss()
# loss = loss_fn(logits.view(-1, logits.shape[-1]), target_ids)

# print("CrossEntropyLoss:", loss.item())
loss

tensor(74.1456)

## Chaning Temperature

In [16]:
shift_logits = logits[..., :-1, :].contiguous()
shift_labels = input_ids[..., 1:].contiguous()
temperature = 1
# Apply temperature scaling to the logits
shift_logits = shift_logits / temperature

# Flatten the tokens
loss_fct = CrossEntropyLoss()
loss_ce = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
loss_ce

tensor(74.1456)

In [17]:
def top_k_filtering(logits, top_k=50):
    """
    Apply top-k filtering to select a reduced set of candidates from logits.
    """
    values, indices = torch.topk(logits, top_k, dim=-1)
    min_values = values[:, -1].unsqueeze(-1).repeat(1, logits.shape[-1])
    logits = torch.where(logits < min_values, torch.tensor(-float('Inf')), logits)
    return logits


In [30]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can use other GPT-2 variants as well
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add the [PAD] token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the top-k value (e.g., 50 for top-50 sampling)
top_k = 50
# Set the number of candidates you want to return
num_candidates = 3

# Define your input text
input_text = "Once upon a time, there was a"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate the next words using top-k sampling
with torch.no_grad():
    logits = model(input_ids).logits[:, -1, :]  # Get the logits for the last word
    # Apply top-k filtering to select a reduced set of candidates
    filtered_logits = top_k_filtering(logits, top_k=top_k)
    # Sample from the filtered candidates, returning num_candidates samples
    next_token_ids = torch.multinomial(torch.softmax(filtered_logits, dim=-1), num_candidates)

# Convert the generated token IDs to words
generated_words = [tokenizer.decode(token_id.item()) for token_id in next_token_ids[0]]

print("Generated Words:", generated_words)

Generated Words: [' great', ' man', ' small']


In [6]:
from utils import privacy_clue_words
def dynamic_temperature(x, privacy_clue_words = privacy_clue_words):
    x = text.split()
    n = len(x)
    m = n - 1 
    xi_index = None
    for i, token in enumerate(x):
        if token in privacy_clue_words:
            xi_index = i
            break
    T, alpha = 1, 0.5
    if xi_index is not None:
        temperature_m = T + alpha * n / abs(xi_index - m)
    else:
        temperature_m = T
    # n, m, xi_index, temperature_m
    return temperature_m

text = "My Password is 555-456 which can unlock everything in my house"
dynamic_temperature(text)

1.6111111111111112

In [1]:
import torch
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)

class argument:
    def __init__(self):
        self.dataset_name = None # 'wikitext'
        # self.dataset_config_name = 'wikitext-2-raw-v1'
        self.output_dir = './logs/' 
        self.seed = 5555
        self.learning_rate = 5e-5
        self.block_size = 1024
        self.do_ref_model = False
        self.hidden_size = 768
        self.beta = 1
        
        self.config_name = None
        self.model_name_or_path = 'gpt2'
        self.tokenizer_name = 'gpt2'
        self.use_slow_tokenizer = False  
        
args = argument()

if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
    
if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

if args.model_name_or_path:
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )
else:
    logger.info("Training new model from scratch")
    model = AutoModelForCausalLM.from_config(config)

model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [2]:
# Add the [PAD] token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

input_texts = "My password is 532-789" 
# Tokenize the input texts and convert them to tensors
input_ids = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)['input_ids']
# Convert input_ids back to words
decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
decoded_tokens

['My', 'Ġpassword', 'Ġis', 'Ġ5', '32', '-', '789']

In [3]:
cleaned_tokens = [token.lstrip('Ġ') for token in decoded_tokens]
cleaned_tokens

['My', 'password', 'is', '5', '32', '-', '789']

In [30]:
# Generate logits from the model
with torch.no_grad():
    outputs = model(input_ids=input_ids, labels =  input_ids)
    logits = outputs.logits
    loss = outputs.loss

In [31]:
dynamic_T = dynamic_temperature(cleaned_tokens)
dynamic_T

1.6111111111111112

In [43]:
#loss1
logits_original = logits #student -> log softmax
#loss2 dynamic temperature
logits_temp = logits #teacher -> softmax

In [44]:
loss

tensor(78.3643)

In [45]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss, KLDivLoss
import torch.nn.functional as F
T = 1
loss_kd = KLDivLoss()
KD_loss = loss_kd(F.log_softmax(logits_temp/dynamic_T, dim=1), F.softmax(logits_original/T, dim=1)) 
print(KD_loss)

alpha = 0.2
total = alpha * KD_loss + (1-alpha) * loss
total

tensor(2.0481e-07)


tensor(62.6915)

In [52]:
# input_ids = tokenizer('telephone number is', return_tensors="pt", padding=True, truncation=True)['input_ids']
# decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
# decoded_tokens

['tele', 'phone', 'Ġnumber', 'Ġis']

In [53]:
# input_ids = tokenizer('my business trip', return_tensors="pt", padding=True, truncation=True)['input_ids']
# decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
# decoded_tokens